In [2]:
# This file aims to do pre-processing to get bug features. The difference with 'filter_data.ipynb' is that is that it uses more fields 
# of the bugs. I process the bugs assigned to CPD platform team and store them in the table 'bugs_cpdplatform_ff' with 'ff' meaning
# 'full fields'
import MySQLdb
import pandas
import re
import itertools
import nltk
# from bs4 import BeautifulSoup

In [2]:
# conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
# cur =conn.cursor()
# sql = '''SELECT bug_id, thetext
# FROM longdescs
# WHERE bug_id = 943195'''
# cur.execute(sql)
# temp = cur.fetchall()
# for item in temp:
#     print len(item[1])
# print temp[4]

In [5]:
# remove cites in a comment
def rm_cite(raw_text):
    lst = []
    iscite = False
    lines = raw_text.splitlines()
    for idx,line in enumerate(lines):
        if line.startswith("(In reply to comment"):
            iscite = True
            lst.append(idx)
        elif line.startswith(">"):
            if iscite == True:
                lst.append(idx)
        else:
            iscite = False
        
    return '\n'.join([item[1] for item in filter(lambda x: x[0] not in lst, enumerate(lines))])

In [6]:
from nltk.stem.snowball import SnowballStemmer
# from bs4 import BeautifulSoup

def preprocess_strict( raw_description, output):
    # Function to convert a raw bug description to a list of words or a cleared string, it's "strict" in the sense that it removes all non-letters 
    # Remove markers
    # text = BeautifulSoup(raw_description).get_text() 
    
    # Remove cites
    description = rm_cite(raw_description)
    
    # Remove urls
    text = re.sub("((mailto\:|(news|(ht|f)tp(s?))\://){1}\S+)", " ", description)
    
    # Remove non-letters        
    # letters_only = re.sub("[^a-zA-Z_/\-\.]", " ", description_text)
    letters_only = re.sub("[^a-zA-Z\.]", " ", text)
    letters_only = re.sub("\.(?!((c|h|cpp|py)\s+$))", " ", letters_only)
    
    # Convert to lower case, tokenize
    words = [word for sent in nltk.sent_tokenize(letters_only.lower()) for word in nltk.word_tokenize(sent)]
    
    # Remove stop words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    meaningful_words = [w for w in words if not w in stopwords]
    
    # Stemming
    snowball = SnowballStemmer("english")
    stems = [snowball.stem(w) for w in meaningful_words]
    
    if output == "list":
        return stems
    else:
        return " ".join(stems)

In [7]:
def preprocess_mild( raw_description, output):
    # Function to convert a raw bug description to a list of words or a cleared string, it's "mild" in the sense that it removes words that don't contain letters
    # Remove markers
    # text = BeautifulSoup(raw_description).get_text() 
    
    # Remove cites
    description = rm_cite(raw_description)
    
    # Remove urls
    text = re.sub("((mailto\:|(news|(ht|f)tp(s?))\://){1}\S+)", " ", description)
    text = re.sub("[^\x00-\x7f]", " ", text)
    
    # Convert to lower case, tokenize
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    
    # Remove stop words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    meaningful_tokens = [token for token in filtered_tokens if not token in stopwords]
    
    # Stemming
    snowball = SnowballStemmer("english")
    stems = [snowball.stem(t) for t in meaningful_tokens]
    
    if output == "list":
        return stems
    else:
        return " ".join(stems)

In [8]:
conn_data = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur_data =conn_data.cursor()

sql = '''SELECT bug_id, assigned_to, short_desc, host_op_sys, guest_op_sys, priority, product_id, category_id, component_id, found_in_product_id, found_in_version_id, found_in_phase_id, cf_security
FROM bugs,profiles 
WHERE bugs.assigned_to=profiles.userid and profiles.login_name in ('hfu','letian','vbhakta','weili','nmukuri','zhoum','hxie','shiyaoy','shanpeic','souravk','vaibhavk','fangchiw','gengshengl')'''

cur_data.execute(sql)
bugs_cpdplatf = cur_data.fetchall()

In [10]:
import itertools

bugdata = []

conn_data = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur_data =conn_data.cursor()

for item in bugs_cpdplatf:
    sql_ld = '''SELECT bug_id, thetext
    FROM longdescs
    WHERE bug_id = (%s)''' % str(item[0])
    
    df = pandas.io.sql.read_sql(sql_ld, conn_data)
    
    keywords = preprocess_strict(item[2],"list")
    
    text = list(df['thetext'].map(lambda x: preprocess_mild(x,"list")))
    
    # judge if a comment is a technical comment, the standard is that the length is bigger than 10 and it has some keywords (which is extracted from the 'short_desc' of 
    # the same bug) or the length is bigger than 30
    def judge(wordlist):       
        if (len(wordlist) > 10 and len(set(keywords) & set(wordlist)) > 0 ) or len(wordlist) > 30:
            return True
        else:
            return False
    
    long_desc = list(itertools.chain(*filter(judge, text)))
    
    sql_ni = '''SELECT bug_id, who
    FROM needinfo
    WHERE bug_id = (%s)''' % str(item[0])
    
    df = pandas.io.sql.read_sql(sql_ni, conn_data)
    
    needinfo = list(df['who'].map(str))
    
    bugdata.append((str(item[0]), str(item[1]), item[2], item[3], item[4], item[5], str(item[6]), str(item[7]), str(item[8]), str(item[9]), str(item[10]), str(item[11]), str(item[12]), " ".join(needinfo), " ".join(keywords), " ".join(long_desc)))
    


In [12]:
conn_feature = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur_feature =conn_feature.cursor()

sql_ins = '''INSERT into bugs_cpdplatform_ff
VAlUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
    
cur_feature.executemany(sql_ins,bugdata)
conn_feature.commit()

In [5]:
conn_data = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur_data =conn_data.cursor()

sql_ni = '''SELECT bug_id, who
FROM needinfo
WHERE bug_id = 1285973'''

df = pandas.io.sql.read_sql(sql_ni, conn_data)
    
needinfo = list(df['who'].map(str))

print needinfo
print df

['19367']
    bug_id    who
0  1285973  19367
